In [2]:
import csv
import requests
from bs4 import BeautifulSoup
import os
import time

import gspread
from oauth2client.service_account import ServiceAccountCredentials

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# List of countries
countries = [
    "Indonesia", "United States", "Hong kong", "Germany", "United Kingdom", "Canada", "Japan", "China",
    "France", "South Korea", "Taiwan", "Australia", "Netherlands", "Singapore",
    "Thailand", "Malaysia", "Belgium", "Philippines", "Turkey", "Vietnam", "Portugal"
]

folder_path = '30%'
if os.path.exists(folder_path):
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            os.remove(file_path)

# Function to scrape and save data for a given country
def scrape_country_data(country):
    # Set up the WebDriver (e.g., ChromeDriver)
    driver = webdriver.Chrome()

    # Construct the URL
    url = f"https://www.investing.com/equities/{country.lower().replace(' ', '-')}/top-stock-gainers"

    # Open the URL in the browser
    driver.get(url)

    try:
        # Wait for the "Performance" tab button to be clickable
        performance_tab = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-test-tab-id='1']"))
        )

        # Click on the "Performance" tab button
        performance_tab.click()

        # Wait for the table to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'datatable_body__tb4jX'))
        )

        # Create the "30" folder if it doesn't exist
        if not os.path.exists('30%'):
            os.mkdir('30%')

        # Open a CSV file inside the "30" folder for writing
        with open(os.path.join('30%', f'{country}.csv'), 'w', newline='', encoding='utf-8') as csvfile:
            # Create a CSV writer
            csv_writer = csv.writer(csvfile)

            # Write header row
            csv_writer.writerow(['Name', '1 Month'])

            # Find all rows in the table
            rows = driver.find_elements(By.CLASS_NAME, 'datatable_body__tb4jX')[0].find_elements(By.TAG_NAME, 'tr')

            # Iterate over rows and write data to CSV
            for row in rows:
                # Extract data from each cell within the row
                cells = row.find_elements(By.TAG_NAME, 'td')
                row_data = [cell.text.strip() for cell in cells]

                # Check if the "1 Month" column is greater than +30.00%
                one_month_percentage = row_data[3].replace('%', '')
                if one_month_percentage and float(one_month_percentage) > 30.00:
                    # Write the row data to the CSV file
                    csv_writer.writerow([row_data[0], row_data[3]])

    finally:
        # Close the browser window
        driver.quit()

# Scrape data for each country
for country in countries:
    scrape_country_data(country)

spreadsheet_id = '1LYn-q4lN1MPu88gf_fN4_cjVP1nxLzLp8KXEI8j4xSk'  # Replace with your Google Sheets document ID

# Define the scope and credentials
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('allcompanies-403612-599ae92083a8.json', scope)

# Authenticate with Google Sheets
client = gspread.authorize(creds)

# Try to open the Google Sheets document using its ID
try:
    spreadsheet = client.open_by_key(spreadsheet_id)
except gspread.SpreadsheetNotFound:
    print("The specified Google Sheets document does not exist. Creating a new one...")
    spreadsheet = client.create('My New Spreadsheet')
    # Share the newly created spreadsheet if needed
    spreadsheet.share('aliskhattak664@gmail.com', perm_type='user', role='writer')  # Replace with your email

# Get a list of all worksheet titles in the Google Sheets document
worksheet_titles = [worksheet.title for worksheet in spreadsheet.worksheets()]

# Iterate through the worksheets and delete them if their title is not "dummy"
for worksheet_title in worksheet_titles:
    if worksheet_title != "dummy":
        worksheet = spreadsheet.worksheet(worksheet_title)
        spreadsheet.del_worksheet(worksheet)

# List all CSV files in the folder
csv_files = [f for f in os.listdir(30%  ) if f.endswith('.csv')]

# Iterate through each CSV file, create a new sheet, and upload its data
for csv_file in csv_files:
    csv_file_path = os.path.join(folder_path, csv_file)
    with open(csv_file_path, 'r') as file:
        content = file.read()
        data = [line.split(',') for line in content.split('\n')]
    sheet_name = os.path.splitext(csv_file)[0]  # Use the CSV filename as the sheet name
    worksheet = spreadsheet.add_worksheet(title=sheet_name, rows="100", cols="20")  # Adjust rows and cols as needed
    worksheet.insert_rows(data, 2)  # Insert data in the second row (change as needed)
    print(f"Data from '{csv_file}' has been uploaded to '{sheet_name}' in the Google Sheets document.")
    time.sleep(3)

print("All CSV files in the folder have been uploaded to the Google Sheets document.")



Data from 'Australia.csv' has been uploaded to 'Australia' in the Google Sheets document.
Data from 'Belgium.csv' has been uploaded to 'Belgium' in the Google Sheets document.
Data from 'Canada.csv' has been uploaded to 'Canada' in the Google Sheets document.
Data from 'China.csv' has been uploaded to 'China' in the Google Sheets document.
Data from 'France.csv' has been uploaded to 'France' in the Google Sheets document.
Data from 'Germany.csv' has been uploaded to 'Germany' in the Google Sheets document.
Data from 'Hong kong.csv' has been uploaded to 'Hong kong' in the Google Sheets document.
Data from 'Indonesia.csv' has been uploaded to 'Indonesia' in the Google Sheets document.
Data from 'Japan.csv' has been uploaded to 'Japan' in the Google Sheets document.
Data from 'Malaysia.csv' has been uploaded to 'Malaysia' in the Google Sheets document.
Data from 'Netherlands.csv' has been uploaded to 'Netherlands' in the Google Sheets document.
Data from 'Philippines.csv' has been uploade